In [2]:
import time 
import requests
import cv2
import operator
import numpy as np
import json

# Import compatibility libraries (python 2/3 support)
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

# Python 3
from urllib.request import urlopen, Request
from urllib.parse import urlparse, urlencode
from http.client import HTTPSConnection

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [16]:
# Variables# Varia 
_url = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v2.0/Prediction/5dc6fe72-933a-4d24-afcc-b00f114c2882/url?iterationId=681acc38-9314-4077-b0d9-5c4e87a9a741'
_key = None #Here you have to paste your primary key
_maxNumRetries = 10

In [24]:
def processRequest( json, data, headers):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers)

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json() ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json() ) )

        break
        
    return result

In [26]:
def renderResultOnImage( result, img ):
    
    """Display the obtained results onto the input image"""

    R = int(result['color']['accentColor'][:2],16)
    G = int(result['color']['accentColor'][2:4],16)
    B = int(result['color']['accentColor'][4:],16)

    cv2.rectangle( img,(0,0), (img.shape[1], img.shape[0]), color = (R,G,B), thickness = 25 )

    if 'categories' in result:
        categoryName = sorted(result['categories'], key=lambda x: x['score'])[0]['name']
        cv2.putText( img, categoryName, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3 )
# URL direction to image
urlImage = 'https://oxfordportal.blob.core.windows.net/vision/Analysis/3.jpg'

# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories'} 

headers = dict()
headers['Prediction-Key'] = 'b4e997f4155f4bf6957aa1196d858c6a'
headers['Content-Type'] = 'application/json' 

json = { 'Url': urlImage } 
data = None

result = processRequest( json, data, headers)
print(result)

{'id': '6a6a942e-35c2-41c6-a04a-90f68287a630', 'project': '5dc6fe72-933a-4d24-afcc-b00f114c2882', 'iteration': '681acc38-9314-4077-b0d9-5c4e87a9a741', 'created': '2018-05-22T06:14:07.0358801Z', 'predictions': [{'probability': 0.00128097238, 'tagId': 'c9fa1210-8cca-4ae9-82d1-3b6bd7033975', 'tagName': 'insulated_jackets'}, {'probability': 0.000229823767, 'tagId': '1163043d-1595-4e98-afcd-2b511ae81a0d', 'tagName': 'hardshell_jackets'}]}
